In [74]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, LSTM, Masking, Dropout, TimeDistributed, Input, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import RootMeanSquaredError



In [59]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 0


In [60]:
def padding(frames, max_frames):
    new_frames = frames
    while len(new_frames) < max_frames:
        zero_array = np.zeros((224,224, 3))
        new_frames.append(zero_array)
    
    return new_frames

In [61]:
def extract_and_resize_frames(video_path, target_size=(224,224), max_frames=120):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
    
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_resized = cv2.resize(frame, target_size)
        frame_resized = frame_resized / 255.0
        frames.append(frame_resized)
    
    #while len(frames) < max_frames:
    #    frames.append(np.zeros((640, 360)))

    frames = padding(frames, 120)
    
    cap.release()
    return np.array(frames)

In [62]:
def create_padding_mask(frames, max_frames=120):
    mask = np.ones((max_frames,))  
    if len(frames) < max_frames:
        mask[len(frames):] = 0  
    return mask

In [63]:
def repeat_rows(df, num_repeats=120):
    repeated_df = pd.DataFrame(np.tile(df.values, (num_repeats, 1)), columns=df.columns)
    
    return repeated_df

In [64]:
def generate_df(X_folder, y_filepath):
    X_filepaths = []
    X_arrays = []
    
    for i in range(35):
        X_filename = f"{X_folder}{i+1}.mp4"
        X_filepaths.append(X_filename)
    
    for filepath in X_filepaths:
        X_array = extract_and_resize_frames(filepath)
        X_arrays.append(X_array)

    y_df = pd.read_csv(y_filepath)
    y_df.drop(columns='filename', axis=1, inplace=True)
    y_df = y_df.head(35)
    
    y_df = repeat_rows(y_df)

    X_arrays = np.array(X_arrays)   
    y_array = y_df.to_numpy()
    
    return X_arrays, y_array

In [65]:
X, y = generate_df("../app/processed_data/", "../model/data/coordinate_data.csv")
X.shape


(35, 120, 224, 224, 3)

In [66]:
def create_feature_extraction_cnn(input_shape):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    
    for layer in base_model.layers:
        layer.trainable = False
        
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu')
    ])
    
    return model

In [67]:
def feature_extracter(X):
    all_features = []
    model = create_feature_extraction_cnn((224, 224, 3))
    model.save_weights('CNN.weights.h5')
    model.save('full_CNN.keras')
    for video in X:
        features = model.predict(video)
        all_features.append(features)
    
    
    
    all_features = np.array(all_features)
    return(all_features)

   


In [68]:
X = feature_extracter(X)
X.shape

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 779ms/stepWARNING:tensorflow:5 out of the last 143 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7433a4b78f40> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step  
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 747ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 899ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 810ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 841ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 827ms/step
4/4 ━

(35, 120, 256)

In [75]:
def custom_loss(y_true, y_pred):
    variance_penalty = K.var(y_pred)  # Penalize low variance
    mse = K.mean(K.square(y_true - y_pred))
    return mse - 0.1 * variance_penalty  # Adjust penalty weight


In [78]:
def build_model(input_shape, X, y):
    y = y.reshape((35,120,2))

    model = Sequential([
        Masking(mask_value=0.0, input_shape=input_shape),
        LSTM(8, activation='tanh', return_sequences=True),
        Dropout(0.5),
        BatchNormalization(),
        #LSTM(256, return_sequences=True),
        #Dropout(0.3),
        #BatchNormalization(),
        #Dense(256, activation='relu'),
        Dense(2)

        
    ])
    
    model.compile(
        optimizer='adam', 
        loss=custom_loss,
        metrics = ['mae']
    )
    
    model.fit(
        X, y,
        epochs=200,
        #callbacks = [checkpoint_callback]
    )
    
    return model

In [79]:
X = (X - np.mean(X)) / np.std(X)
model = build_model(input_shape=(120, 256), X=X, y=y)

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - loss: 9.0868 - mae: 2.2021
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 8.9486 - mae: 2.1634
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 8.8900 - mae: 2.1141
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 8.6179 - mae: 2.0653
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 8.4878 - mae: 2.0149
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 8.3067 - mae: 1.9622
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 8.1716 - mae: 1.9531
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 8.1446 - mae: 1.9433
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 8.0825 - mae: 1.9278
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 8.0475 - mae: 1.9137
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 7.9576 - mae: 1.9007
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 7.9748 - mae: 1.8986
Epoch 13/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/ste

In [71]:
model.save_weights('LSTM.weights.h5')
model.save('full_LSTM.keras')

In [72]:
#how to load model into API
'''
from tensorflow.keras.models import load_model
model = load_model('full_model.keras')
model.load_weights('model_weights.h5')
'''

"\nfrom tensorflow.keras.models import load_model\nmodel = load_model('full_model.keras')\nmodel.load_weights('model_weights.h5')\n"

In [73]:
model.predict(X)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step


array([[[1.668782  , 0.32522807],
        [1.6676788 , 0.32480317],
        [1.6689026 , 0.3250353 ],
        ...,
        [1.6719756 , 0.32409832],
        [1.6719756 , 0.32409832],
        [1.6719756 , 0.32409832]],

       [[1.67255   , 0.32501516],
        [1.6730452 , 0.3248096 ],
        [1.6711283 , 0.3250797 ],
        ...,
        [1.6719756 , 0.32409832],
        [1.6719756 , 0.32409832],
        [1.6719756 , 0.32409832]],

       [[1.6719987 , 0.32416344],
        [1.6675003 , 0.3235767 ],
        [1.6712086 , 0.3264726 ],
        ...,
        [1.6719756 , 0.32409832],
        [1.6719756 , 0.32409832],
        [1.6719756 , 0.32409832]],

       ...,

       [[1.7102313 , 0.3567561 ],
        [1.674537  , 0.32596993],
        [1.6682239 , 0.32168576],
        ...,
        [1.6719756 , 0.32409832],
        [1.6719756 , 0.32409832],
        [1.6719756 , 0.32409832]],

       [[1.6686518 , 0.32472903],
        [1.6707911 , 0.3240525 ],
        [1.6715906 , 0.3246472 ],
        .